In [ ]:
from time import time
from pyspark import SparkContext
for j in range(1,10):
    sc = SparkContext(master=f'local[{j}]')
    t0 = time()
    for i in range(10):
        sc.parallelize([1,2] * 1000000).reduce(lambda x,y:x+y)
    print(f'{j} executors, time= {time() - t0}')
    sc.stop()

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType,IntegerType,StructType 
from pyspark.sql.functions import collect_list, split, regexp_replace, col, round,concat,lit,avg
from pyspark.sql.functions import avg, col, collect_list, concat, lit, regexp_replace, round, split, struct, UserDefinedFunction

In [2]:
# Importing packages	
import pyspark	
from pyspark.sql import SparkSession	
from pyspark.sql.window import Window	
from pyspark.sql.functions import dense_rank	
from pyspark.sql.functions import percent_rank	

In [3]:
spark = SparkSession.builder.master("local[*]").appName('link-mast').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/05 04:49:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/05 04:49:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
commToCand = spark.read.format("csv").options(delimiter='|',inferschema='true',header='true').load('data/itpas2.txt')
commToCand = commToCand.drop("AMNDT_IND","RPT_TP","IMAGE_NUM","ENTITY_TP","CITY","ZIP_CODE","TRANSACTION_PGI","TRANSACTION_TP","EMPLOYER","OCCUPATION","TRANSACTION_DT","TRAN_ID","FILE_NUM","MEMO_CD","MEMO_TEXT","SUB_ID")
commToCand.show(5)

+---------+--------------------+-----+---------------+---------+---------+
|  CMTE_ID|                NAME|STATE|TRANSACTION_AMT| OTHER_ID|  CAND_ID|
+---------+--------------------+-----+---------------+---------+---------+
|C00761528|GEORGIANS FOR KEL...|   GA|           2000|C00729608|S0GA00526|
|C00761528|   PERDUE FOR SENATE|   GA|           2000|C00547570|S4GA11285|
|C00364471|FRIENDS OF DICK D...|   IL|           2500|C00148999|S6IL00151|
|C00364471|FRIENDS OF DICK D...|   IL|          -2500|C00148999|S6IL00151|
|C00710780| WARNOCK FOR GEORGIA|   GA|           1500|C00736876|S0GA00559|
+---------+--------------------+-----+---------------+---------+---------+
only showing top 5 rows



In [5]:
commMast = spark.read.format("csv").options(delimiter='|',inferschema='true',header='true').load('data/cm(1).txt')
commMast = commMast.drop("CMTE_ST1","CMTE_ST2","CMTE_ZIP","CMTE_FILING_FREQ","CAND_ID")
commMast.show(5)

+---------+--------------------+--------------------+-------------+-------+---------+-------+--------------------+------+--------------------+
|  CMTE_ID|             CMTE_NM|             TRES_NM|    CMTE_CITY|CMTE_ST|CMTE_DSGN|CMTE_TP|CMTE_PTY_AFFILIATION|ORG_TP|    CONNECTED_ORG_NM|
+---------+--------------------+--------------------+-------------+-------+---------+-------+--------------------+------+--------------------+
|C00000059|HALLMARK CARDS, I...|   KLEIN, CASSIE MS.|  KANSAS CITY|     MO|        B|      Q|                 UNK|     C|HALLMARK CARDS, INC.|
|C00000422|AMERICAN MEDICAL ...|   WALKER, KEVIN MR.|   WASHINGTON|     DC|        B|      Q|                   -|     -| ALABAMA MEDICAL PAC|
|C00000489|D R I V E POLITIC...|       JERRY SIMS JR|OKLAHOMA CITY|     OK|        U|      N|                   -|     L|                   -|
|C00000547|KANSAS MEDICAL SO...|C. RICHARD BONEBR...|       TOPEKA|     KS|        U|      Q|                 UNK|     T|                   -|

In [6]:
commToCand.createOrReplaceTempView("tempcommToCand")
commMast.createOrReplaceTempView("tempCommMast")

In [71]:
commMast.coalesce(1).write.format('json').save('out_cmteMast_clean')

In [8]:
table1 = spark.sql("""
SELECT CMTE_ID, OTHER_ID, SUM(TRANSACTION_AMT) AS TOTAL_DONATED
FROM tempcommToCand
GROUP BY CMTE_ID, OTHER_ID
ORDER BY CMTE_ID asc, SUM(TRANSACTION_AMT) DESC
""")
table1.show(10)
table1.createOrReplaceTempView("temp_table1")
table1.dtypes

+---------+---------+-------------+
|  CMTE_ID| OTHER_ID|TOTAL_DONATED|
+---------+---------+-------------+
|C00000059|C00670034|         5000|
|C00000059|C00395848|         5000|
|C00000059|C00718346|         5000|
|C00000059|C00359034|         5000|
|C00000059|C00326801|         2500|
|C00000059|C00540732|         2500|
|C00000059|C00704981|         2000|
|C00000059|C00458679|         2000|
|C00000422|C00409581|        10000|
|C00000422|C00468256|        10000|
+---------+---------+-------------+
only showing top 10 rows



[('CMTE_ID', 'string'), ('OTHER_ID', 'string'), ('TOTAL_DONATED', 'bigint')]

In [9]:
table2 = spark.sql("""

    select CMTE_ID, 
           OTHER_ID,
           TOTAL_DONATED, 
           row_number() over (partition by CMTE_ID order by TOTAL_DONATED desc) as donation_rank         
    from temp_table1
    order by CMTE_ID desc, donation_rank asc
    

    
""")
table2.show(20)
table2.count()

+---------+---------+-------------+-------------+
|  CMTE_ID| OTHER_ID|TOTAL_DONATED|donation_rank|
+---------+---------+-------------+-------------+
|C90021668|H2OH09031|         5471|            1|
|C90021650|H2OH13264|         3897|            1|
|C90021635|S6PA00274|         9629|            1|
|C90021619|S2OH00436|         3907|            1|
|C90021569|S2PA00638|        48595|            1|
|C90021551|H2NM02191|        51472|            1|
|C90021544|H2NY10324|        10000|            1|
|C90021536|S0GA00559|        60588|            1|
|C90021536|S6PA00274|        52714|            2|
|C90021536|S2OH00402|        42226|            3|
|C90021536|S6NV00200|        32562|            4|
|C90021494|H8MI09118|         2500|            1|
|C90021452|S6NV00200|       540687|            1|
|C90021452|H2NV04011|       117059|            2|
|C90021452|H8NV03036|        96323|            3|
|C90021452|H6NV04020|        71102|            4|
|C90021411|H2AZ09241|         1582|            1|


118763

In [10]:
table2.createOrReplaceTempView("CommToAllCandRanked")

In [11]:
candMast = spark.read.format("csv").options(delimiter='|',inferschema='true',header='true').load('data/cn(1).txt')
candMast = candMast.drop("CAND_ICI","CAND_ST1","CAND_ST2","CAND_CITY","CAND_ST","CAND_ZIP")
candMast.show(5)

+---------+--------------------+--------------------+----------------+--------------+-----------+--------------------+-----------+---------+
|  CAND_ID|           CAND_NAME|CAND_PTY_AFFILIATION|CAND_ELECTION_YR|CAND_OFFICE_ST|CAND_OFFICE|CAND_OFFICE_DISTRICT|CAND_STATUS| CAND_PCC|
+---------+--------------------+--------------------+----------------+--------------+-----------+--------------------+-----------+---------+
|H0AK00105|        LAMB, THOMAS|                 NNE|            2020|            AK|          H|                   -|          N|C00607515|
|H0AL01055| CARL, JERRY LEE, JR|                 REP|            2022|            AL|          H|                  01|          C|C00697789|
|H0AL01063|LAMBERT, DOUGLAS ...|                 REP|            2020|            AL|          H|                  01|          P|C00701557|
|H0AL01097|     AVERHART, JAMES|                 DEM|            2020|            AL|          H|                  01|          P|C00708867|
|H0AL02087|  

In [12]:
candMast.createOrReplaceTempView("tempcandMast")
# candMast.write.saveAsTable("tbl_candMast")

In [13]:
test = spark.sql("""
SELECT * 
FROM tempcandMast 
ORDER BY CAND_OFFICE_DISTRICT ASC
""")
test.show(5)

+---------+--------------------+--------------------+----------------+--------------+-----------+--------------------+-----------+---------+
|  CAND_ID|           CAND_NAME|CAND_PTY_AFFILIATION|CAND_ELECTION_YR|CAND_OFFICE_ST|CAND_OFFICE|CAND_OFFICE_DISTRICT|CAND_STATUS| CAND_PCC|
+---------+--------------------+--------------------+----------------+--------------+-----------+--------------------+-----------+---------+
|H0CA06170|     BISH, CHRISTINE|                 REP|            2022|            CA|          H|                   -|          C|C00721548|
|H0DC01015|      KRUCOFF, DAVID|                 IND|            2020|            DC|          H|                   -|          P|C00719930|
|H0DC00058|NORTON, ELEANOR H...|                 DEM|            2022|            DC|          H|                   -|          C|C00244335|
|H0AS01016|CRICHTON, ORETA T...|                 DEM|            2020|            AS|          H|                   -|          P|C00755843|
|H0DC00165|  

In [14]:
table3 = spark.sql("""
SELECT CommToAllCandRanked.CMTE_ID, CommToAllCandRanked.OTHER_ID, CommToAllCandRanked.TOTAL_DONATED, CommToAllCandRanked.donation_rank, 
tempcandMast.CAND_ID, tempcandMast.CAND_NAME, tempcandMast.CAND_PCC, tempcandMast.CAND_PTY_AFFILIATION, tempcandMast.CAND_OFFICE_ST, 
tempcandMast.CAND_OFFICE, tempcandMast.CAND_OFFICE_DISTRICT
FROM CommToAllCandRanked
JOIN tempcandMast
ON (CommToAllCandRanked.OTHER_ID = tempcandMast.CAND_PCC OR CommToAllCandRanked.OTHER_ID = tempcandMast.CAND_ID)
ORDER BY CommToAllCandRanked.CMTE_ID desc, CommToAllCandRanked.donation_rank asc
""")
table3.show(20)

+---------+---------+-------------+-------------+---------+--------------------+---------+--------------------+--------------+-----------+--------------------+
|  CMTE_ID| OTHER_ID|TOTAL_DONATED|donation_rank|  CAND_ID|           CAND_NAME| CAND_PCC|CAND_PTY_AFFILIATION|CAND_OFFICE_ST|CAND_OFFICE|CAND_OFFICE_DISTRICT|
+---------+---------+-------------+-------------+---------+--------------------+---------+--------------------+--------------+-----------+--------------------+
|C90021668|H2OH09031|         5471|            1|H2OH09031|       KAPTUR, MARCY|C00154625|                 DEM|            OH|          H|                  09|
|C90021650|H2OH13264|         3897|            1|H2OH13264|       SYKES, EMILIA|C00801274|                 DEM|            OH|          H|                  13|
|C90021635|S6PA00274|         9629|            1|S6PA00274|FETTERMAN, JOHN KARL|C00765800|                 DEM|            PA|          S|                   -|
|C90021619|S2OH00436|         3907|     

In [15]:
table3.createOrReplaceTempView("commToCandMerged")

In [16]:
table4 = spark.sql("""
select * 
from commToCandMerged
order by CAND_OFFICE_DISTRICT asc

""")
table4.show(20)

+---------+---------+-------------+-------------+---------+--------------------+---------+--------------------+--------------+-----------+--------------------+
|  CMTE_ID| OTHER_ID|TOTAL_DONATED|donation_rank|  CAND_ID|           CAND_NAME| CAND_PCC|CAND_PTY_AFFILIATION|CAND_OFFICE_ST|CAND_OFFICE|CAND_OFFICE_DISTRICT|
+---------+---------+-------------+-------------+---------+--------------------+---------+--------------------+--------------+-----------+--------------------+
|C00000422|C00473371|         2000|          117|S0ND00093|        HOEVEN, JOHN|C00473371|                 REP|            ND|          S|                   -|
|C00000729|C00476317|         5000|           73|S0AR00150|  BOOZMAN, SEN. JOHN|C00476317|                 REP|            AR|          S|                   -|
|C00000422|C00458315|         2000|          120|S0KS00091|        MORAN, JERRY|C00458315|                 REP|            KS|          S|                   -|
|C00000059|C00540732|         2500|     

In [59]:
table4.count()

122332

In [60]:
type(table3)
table3.printSchema()

root
 |-- CMTE_ID: string (nullable = true)
 |-- OTHER_ID: string (nullable = true)
 |-- TOTAL_DONATED: long (nullable = true)
 |-- donation_rank: integer (nullable = false)
 |-- CAND_ID: string (nullable = true)
 |-- CAND_NAME: string (nullable = true)
 |-- CAND_PCC: string (nullable = true)
 |-- CAND_PTY_AFFILIATION: string (nullable = true)
 |-- CAND_OFFICE_ST: string (nullable = true)
 |-- CAND_OFFICE: string (nullable = true)
 |-- CAND_OFFICE_DISTRICT: string (nullable = true)



In [61]:
#get the comittee id
cmteId = spark.sql("""
SELECT DISTINCT CMTE_ID
FROM commToCandMerged
""")

noCmteId = spark.sql("""
SELECT CMTE_ID, CAND_NAME, OTHER_ID, TOTAL_DONATED, donation_rank, CAND_ID, CAND_PCC, CAND_PTY_AFFILIATION, CAND_OFFICE_ST, CAND_OFFICE, CAND_OFFICE_DISTRICT
FROM commToCandMerged
""")

cmteTable = cmteId.join(
    noCmteId
        .groupBy("CMTE_ID")
        .agg(collect_list(struct(noCmteId.CAND_NAME, noCmteId.OTHER_ID, noCmteId.TOTAL_DONATED, noCmteId.donation_rank, noCmteId.CAND_ID, noCmteId.CAND_PCC
                                , noCmteId.CAND_PTY_AFFILIATION, noCmteId.CAND_OFFICE_ST, noCmteId.CAND_OFFICE, noCmteId.CAND_OFFICE_DISTRICT)).alias("committeeTable"))
    , "CMTE_ID"
    , "outer"
)

cmteTable.printSchema()

root
 |-- CMTE_ID: string (nullable = true)
 |-- committeeTable: array (nullable = true)
 |    |-- element: struct (containsNull = false)
 |    |    |-- CAND_NAME: string (nullable = true)
 |    |    |-- OTHER_ID: string (nullable = true)
 |    |    |-- TOTAL_DONATED: long (nullable = true)
 |    |    |-- donation_rank: integer (nullable = false)
 |    |    |-- CAND_ID: string (nullable = true)
 |    |    |-- CAND_PCC: string (nullable = true)
 |    |    |-- CAND_PTY_AFFILIATION: string (nullable = true)
 |    |    |-- CAND_OFFICE_ST: string (nullable = true)
 |    |    |-- CAND_OFFICE: string (nullable = true)
 |    |    |-- CAND_OFFICE_DISTRICT: string (nullable = true)



In [62]:
cmteTable.coalesce(1).write.format('json').save('out_cmte_infoPageRaw')